In [2]:
import numpy as np
import os
from pathlib import Path
import mediapipe as mp
import cv2
import random
from PIL import Image
from typing import Tuple
import csv
import pickle
from tqdm import tqdm
mp_pose = mp.solutions.pose

current_working_dir = os.getcwd()
GENERAL_DATA_PATH = Path(current_working_dir) / 'data'
DATA_PATH = GENERAL_DATA_PATH

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b) # middle
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if(angle > 180):
        angle = 360 - angle

    return angle

In [19]:
DATA_PATH = Path(current_working_dir)

def get_image_numpy(image_file_name : str) -> np.ndarray:
    return np.array(Image.open(DATA_PATH / 'Train' / image_file_name))

def get_random_image_numpy() -> np.ndarray:
    images = os.listdir(DATA_PATH / 'Train')
    random_image_file_name = random.choice(images)
    return get_image_numpy(random_image_file_name)

class PoseExtractor():
    extraction_output_len = 11
    empty_extraction = [None for _ in range(4 * extraction_output_len)]
    
    def __init__(
            self,
            source_data_path : str,
            destination_data_path : str,
            model_complexity : int,
            min_detection_confidence : float = 0.5
            ) -> None:
        self.keypoints_names = ["left_elbow", "left_shoulder", "left_knee", "right_elbow", "right_shoulder", "right_knee", "body1", "body2", "body3", "body4"]
        self.source_data_path = Path(source_data_path)
        self.destination_data_path = Path(destination_data_path)
        self.pose = mp.solutions.pose.Pose(
            static_image_mode=True,
            model_complexity=model_complexity,
            smooth_landmarks=True,
            enable_segmentation=False,
            smooth_segmentation=False,
            min_detection_confidence=min_detection_confidence,
            min_tracking_confidence=0.5
        )
    
    @property
    def columns_names(self) -> np.ndarray:
        return self.keypoints_names

    def load_image_as_ndarray(
            self,
            image_file_name : str,
            train : bool
            ) -> np.ndarray:
        if train:
            dataset_type = 'Train'
        else:
            dataset_type = 'Test'
        
        image = np.array(Image.open(self.source_data_path / dataset_type / image_file_name))

        return image

    def extract_pose(
            self,
            image: np.ndarray
            ) -> Tuple:
        pose_results = self.pose.process(image).pose_landmarks
        if pose_results is None:
            return None

        landmarks = pose_results.landmark        

        # Calculate angles

        ## Angle 11-13-15
        left_elbow = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y])

        ## Angle 13-11-23
        left_shoulder = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])

        ## Angle 23-25-27
        left_knee = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y])

        ## Angle 12-14-16
        right_elbow = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y])

        ## Angle 14-12-24
        right_shoulder = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y])

        ## Angle 24-26-28
        right_knee = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y])

        ## Angle 12-11-23
        body1 = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])

        ## Angle 11-12-24
        body2 = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y])

        ## Angle 11-23-24
        body3 = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y])

        ## Angle 12-24-23
        body4 = calculate_angle(
            [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
            [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
            [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y])

        return [left_elbow, left_shoulder, right_elbow, right_shoulder, left_knee, right_knee, body1, body2, body3, body4]
    
    def gray_to_rgb(self, image: np.ndarray) -> np.ndarray:
        return cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

    def extract_poses_and_write_to_csv(
            self,
            save_file_name : str,
            sample : bool,
            train : bool = True,
            n_samples : int = 1,
            delimiter : str = ';'
            ) -> None:
        if train:
            images = os.listdir(self.source_data_path / 'Train')
        else:
            images = os.listdir(self.source_data_path / 'Test')
        
        if sample:
            images = random.sample(images, n_samples)
        
        with open(self.destination_data_path / save_file_name, 'w', newline='') as write_file:
            writer = csv.writer(write_file, delimiter=delimiter)
            writer.writerow(['name'] + self.columns_names)
            
            data_rows = []
            
            for image_filename in tqdm(images):
                image = self.load_image_as_ndarray(image_filename, train)
                name = os.path.basename(image_filename)
                
                if len(image.shape) < 3:
                    image = self.gray_to_rgb(image)
                extracted_pose = None
                
                if image.shape[2] == 3:
                    extracted_pose = self.extract_pose(image)

                if extracted_pose is not None:
                    data_rows.append([name] + extracted_pose)  # Include the image filename in the row
                else:
                    data_rows.append([name] + self.empty_extraction)
            
            writer.writerows(data_rows)

In [20]:
pose_extractor = PoseExtractor(
    source_data_path=DATA_PATH,
    destination_data_path=GENERAL_DATA_PATH,
    model_complexity=1
)

In [15]:
data_rows = pose_extractor.extract_poses_and_write_to_csv(
    sample=False,
    train=True,
    save_file_name='angles-train.csv',
    )

100%|██████████| 12690/12690 [22:16<00:00,  9.50it/s] 


In [22]:
data_rows = pose_extractor.extract_poses_and_write_to_csv(
    sample=False,
    train=False,
    save_file_name='angles-test.csv',
)

100%|██████████| 4232/4232 [06:30<00:00, 10.85it/s]
